In [1]:
import csv
import gensim
import numpy as np
import jieba
import collections
from gensim.models import word2vec
from sklearn.cluster import KMeans
from openpyxl.workbook import Workbook
from collections import Counter

In [10]:
pip show sklearn-cluster

Note: you may need to restart the kernel to use updated packages.


In [5]:
pip show openpyxl

Name: openpyxl
Version: 3.0.9
Summary: A Python library to read/write Excel 2010 xlsx/xlsm files
Home-page: https://openpyxl.readthedocs.io
Author: See AUTHORS
Author-email: charlie.clark@clark-consulting.eu
License: MIT
Location: c:\users\11874\anaconda3\lib\site-packages
Requires: et-xmlfile
Required-by: Orange3
Note: you may need to restart the kernel to use updated packages.


In [8]:
pip show jieba

Name: jieba
Version: 0.42.1
Summary: Chinese Words Segmentation Utilities
Home-page: https://github.com/fxsjy/jieba
Author: Sun, Junyi
Author-email: ccnusjy@gmail.com
License: MIT
Location: c:\users\11874\anaconda3\lib\site-packages
Requires: 
Required-by: 
Note: you may need to restart the kernel to use updated packages.


In [83]:
import json
with open(r'E:\13wTDI数据\高中低\三元组\triples-low.json','r',encoding='utf-8')as fp:
    emotion_pairs = json.load(fp)

In [84]:
print(len(emotion_pairs))
print(emotion_pairs[len(emotion_pairs)-1])

68288
[]


In [85]:
del(emotion_pairs[len(emotion_pairs)-1])
print(len(emotion_pairs))

68287


In [86]:
path = r"E:\13w聚类结果\ordered2.csv"
with open(path, 'r', encoding = 'gbk') as csvfile:
    reader = csv.reader(csvfile)
    rows = [row for row in reader]
    
for c in rows:
    while '' in c:
        c.remove('')

In [87]:
print(len(rows))
print(rows[0])

7
['1', '服务', '服务态度', '用户已', '态度', '服务人员', '前台', '服务员', '工作人员', '推荐', '配套', '打扫', '前台态度', '管理', '前台接待', '服务质量', '服务设施', '素质', '用户', '服务环境', '服務', '员工', '接待', '效率', '客服', '整体服务', '真心', '服务意识', '阿姨', '前台人员', '办事', '服务生', '接送机', '速度', '客户', '时间', '姑娘', '服務態度', '办理', '客服态度', '退房', '办理入住', '清洁', '前台接待态度', '客服人员', '领导', '服务细节', '入住手续', '前台接待人员', '升级', '服务效率', '服务生态度', '办理速度', '客房服务', '服务水平', '前台效率', '接送', '送机', '带孩子', '总体服务', '接机', '办事效率', '办理手续', '感到', '卫生管理', '客服人员态度', '防疫措施', '入住效率', '前台员工', '员工态度', '进出', '保洁', '態度', '服务管理', '客户体验', '入住的', '业务', '商务人士', '接待人员', '消毒', '办理的速度', '工作态度', '阿姨态度', '维护', '手续', '自动化程度', '管理水平', '送餐', '人员', '实施', '服务优质', '行李', '前台小妹', '机场接送', '店', '查房', '大方', '员工素质', '服务速度', '保洁人员', '服务体验', '接送机师傅', '安全措施', '服务素质', '管理人员', '升级房型', '前台速度', '客户反馈', '换了房型', 'OK', '平台', '吵闹', '前台人', '保安', '安保措施', '入住流程', '服务台态度', '整体服务态度', '前台素质', '笑容', '待客', '总台服务', '治安', '客户说', '携程', '推介', '保洁员', '服务器', '语气', '年纪', '清理', '公司', '店员', '交流', '拿行李', '保安态度', '服务设置', '总台服务员', '办理业务', '前台妹

In [88]:
for c in rows:
    del c[0]

In [89]:
import pandas as pd
import json
def open_dict(Dict='hahah',path='D:/kmeans/kmeans聚类数据及代码/词典/'):
    path = path + '%s.txt' %Dict
    dictionary = open(path, 'r', encoding='utf-8-sig',errors='ignore')
    dict = []
    for word in dictionary:
        word=word.strip('\n')
        word=word.strip(' ')
        dict.append(word)
    return dict

In [90]:
posdict = open_dict(Dict='posdict')#积极情感词典
negdict = open_dict(Dict='negdict')#消极情感词典
inversedict = open_dict(Dict='inversedict')  #修饰词典，同下
mostdict = open_dict(Dict='mostdict')
verydict= open_dict(Dict='verydict')
moredict = open_dict(Dict='moredict')
ishdict = open_dict(Dict='ishdict')
insufficientdict = open_dict(Dict='insufficientdict')

f=open(r'D:\kmeans\kmeans聚类数据及代码\词典\酒店情感词典.txt','r',encoding='utf-8')
words = []
value=[]
for word in f.readlines():
    words.append(word.split(' ')[0])
    value.append(float(word.split(' ')[1].strip('\n')))

c={'words':words,
   'value':value}
fd=pd.DataFrame(c)
pos=fd['words'][fd.value>0]
posdict=posdict+list(pos)    ##加入酒店相关的正向情感词
neg=fd['words'][fd.value<0]
negdict=negdict+list(neg)    ##加入酒店相关的负向情感词
f.close()

In [91]:
fr1 = open(r'D:\kmeans\评论情感个数\pos.txt', 'r', encoding='utf-8') 
P = []
for line in fr1:
    P.append(line.strip())
fr1.close()

fr2 = open(r'D:\kmeans\评论情感个数\neg.txt', 'r', encoding='utf-8') 
N = []
for line in fr2:
    N.append(line.strip())
fr2.close()

In [92]:
def is_inverse(xiu_ci):   ###判断修饰词列表中否定词的个数
    a = 0
    for i in xiu_ci:
        if i in inversedict:
            a += 1
    return a

def get_intensity(xiu_ci):  ###获取修饰词强度的列表
    intensity_list = []
    for i in xiu_ci:
        if(i in inversedict):
            intensity_list.append(-1)
        elif(i in mostdict):
            intensity_list.append(2)
        elif(i in verydict):
            intensity_list.append(1.5)
        elif(i in moredict):
            intensity_list.append(1.2)
        elif(i in ishdict):
            intensity_list.append(0.8)
        elif(i in insufficientdict):
            intensity_list.append(0.5)
        else:
            intensity_list.append(1)    #修饰词不在目前的修饰词表中,赋值1
    return intensity_list

def add_xiu(xiu_ci):    #修饰词的情感强度计算（修饰词包含 强度>0的程度副词和强度=0的否定词）
    v = 1
    if(len(xiu_ci)==1):    #1 如果三元组中只有一个修饰词，则直接取该修饰词的强度；
        for i in xiu_ci:
            if(i in inversedict):
                v = v*(-1)
            elif(i in mostdict):
                v = v*2
            elif(i in verydict):
                v = v*1.5
            elif(i in moredict):
                v = v*1.2
            elif(i in ishdict):
                v = v*0.8
            elif(i in insufficientdict):
                v = v*0.5
    elif(len(xiu_ci)>1):      #2 如果三元组中出现多个修饰词，首先判断程度副词个数
        if((len(xiu_ci)-is_inverse(xiu_ci))==0):     #2.1 程度副词个数=0，表明全为否定词，强度取 (-1)^否定词个数
            v = (-1)**len(xiu_ci)    ##次方运算**
        elif((len(xiu_ci)-is_inverse(xiu_ci))==1):    #2.2 程度副词个数=1
            if(xiu_ci[0] in inversedict):     #2.2.1第一个修饰词为否定词,强度取 (-1)^否定词个数*修饰词的倒数
                for i in xiu_ci[1:]:
                    if i in mostdict:
                        v = v*1/2
                    elif i in verydict:
                        v = v*1/1.5
                    elif i in moredict:
                        v = v*1/1.2
                    elif i in ishdict:
                        v = v*1/0.8
                    elif i in insufficientdict:
                        v = v*1/0.5
                v = v*((-1)**is_inverse(xiu_ci))
            else:                            #2.2.2 第一个修饰词不为否定词,强度取 (-1)^否定词个数*修饰词
                for i in xiu_ci:
                    if i in inversedict:
                        v = v*(-1)
                    elif i in mostdict:
                        v = v*2
                    elif i in verydict:
                        v = v*1.5
                    elif i in moredict:
                        v = v*1.2
                    elif i in ishdict:
                        v = v*0.8
                    elif i in insufficientdict:
                        v = v*0.5
        elif((len(xiu_ci)-is_inverse(xiu_ci))>1):    #2.3 程度副词个数>1
            if(xiu_ci[0] in inversedict):     #2.3.1第一个修饰词为否定词,强度取 (-1)^否定词个数*修饰词的倒数中最大值*0.8
                v = ((-1)**is_inverse(xiu_ci))*max([1/i for i in get_intensity(xiu_ci)])*0.8
            else:                            #2.3.2 第一个修饰词不为否定词,强度取 (-1)^否定词个数*修饰词中最大值*1.8
                v = ((-1)**is_inverse(xiu_ci))*max(get_intensity(xiu_ci))*1.8
    return v

In [93]:
words[0].strip('\ufeff')
words.append('涨')   ## 自定义有关价格"涨"的情感值
value.append(-0.591413615807158)

In [94]:
emotion_value = []    ##三元组情感值
for i,y1 in enumerate(emotion_pairs):
    emotion_value.append([])
    
for i,y1 in enumerate(emotion_pairs):
    for j,y2 in enumerate(y1):
        emotion_value[i].append(0)#每条三元组数

In [95]:
print(len(emotion_value))

68287


In [96]:
def qingxiang(li):    ##情感倾向
    v = 1
    if((li in negdict) | (li in N)):
        v = -1
    return v

In [97]:
dic_qg = dict(zip(words,value))#字典
uncertain = []

for i,y1 in enumerate(emotion_pairs):
    for j,y2 in enumerate(y1):
        if(len(y2['属性词'])!= 0):
            if((len(y2['情感词'])!=0) or (len(y2['修饰词'])!=0)):
                if(len(y2['情感词'])!= 0):
                    emotion_value[i][j]=format(qingxiang(y2['情感词'][0])*(add_xiu(y2['修饰词'])), '.2f')#保留小数位，可更改
                else:
                    emotion_value[i][j]=format(add_xiu(y2['修饰词']), '.2f')#无情感词
            
for i,y1 in enumerate(emotion_value):
    for j,y2 in enumerate(y1):
        try:
            emotion_value[i][j]=eval(emotion_value[i][j])
        except:
            continue

In [98]:
qg = []    #个数
for i in range(len(emotion_pairs)):
    qg.append([0]*len(rows))    #依据主题词不同改数字(类数)

for i,y1 in enumerate(emotion_pairs):
    for j,y2 in enumerate(y1):
        for k in range(len(rows)):
            if(len(y2['属性词'])!=0):
                if(y2['属性词'][0] in rows[k]):
                    if((len(y2['情感词'])!=0) or (len(y2['修饰词'])!=0)):
                        qg[i][k]= qg[i][k]+1

In [99]:
value = []    ##每句每一类累加情感值
for i in range(len(emotion_pairs)):
    value.append([0]*len(rows))
    
for i,y1 in enumerate(emotion_pairs):
    for j,y2 in enumerate(y1):
        if(emotion_value[i][j]!=0):
            if(len(y2['属性词'])!= 0):
                for k in range(len(rows)):
                    if(y2['属性词'][0] in rows[k]):
                        if((len(y2['情感词'])!=0) or (len(y2['修饰词'])!=0)):
                            if(qg[i][k]>0):
                                value[i][k]=value[i][k]+emotion_value[i][j]

In [100]:
print(len(qg))
print(len(value))

68287
68287


In [101]:
for i,y1 in enumerate(value):
    for j,y2 in enumerate(y1):
        if(y2!=0):
            value[i][j]=float(('%.1f' % y2))    ##更改保留小数位

In [102]:
value_sort = [[],[],[],[],[],[],[]]
for i in value:
    value_sort[0].append(i[0])
    value_sort[1].append(i[1])
    value_sort[2].append(i[2])
    value_sort[3].append(i[3])
    value_sort[4].append(i[4])
    value_sort[5].append(i[5])
    value_sort[6].append(i[6])

In [103]:
qg_sort = [[],[],[],[],[],[],[]]
for i in qg:
    qg_sort[0].append(i[0])
    qg_sort[1].append(i[1])
    qg_sort[2].append(i[2])
    qg_sort[3].append(i[3])
    qg_sort[4].append(i[4])
    qg_sort[5].append(i[5])
    qg_sort[6].append(i[6])

## value

In [104]:
v = [[],[],[],[],[],[],[]]
for i in range(len(v)):
    v[i] = sum(value_sort[i])
print(v)
print(sum(v))

[37584.79999999986, 4801.499999999969, 34553.89999999978, 55004.200000001, 10927.099999999982, 4461.10000000003, 998.5]
148331.10000000062


## pop

In [105]:
v1 = [[],[],[],[],[],[],[]]
for i in range(len(v1)):
    v1[i] = sum(qg_sort[i])
print(v1)
print(sum(v1))

[31335, 4907, 37979, 48631, 11242, 5764, 1281]
141139


In [323]:
v1 = [31335,4907,37979,48631,11242,5764,1281]
del(v1[4])
del(v1[0])
v1 = [v1]
v1_nor = preprocessing.normalize(v1, norm='l2')
print(v1_nor)

[[0.07891636 0.61079366 0.78210344 0.09269898 0.02060156]]


In [218]:
h_pop = [0.37555076,0.0487125,0.41356611,0.79363438,0.18078601,0.15108119,0.01368041]
m_pop = [0.51349178,0.07025398,0.4719613,0.67658327,0.15278792,0.16527869,0.01473067]
l_pop = [0.44427431,0.06957249,0.53847435,0.68950068,0.15939147,0.08172322,0.01816229]

h_qg = [0.48238547,0.17325946,0.40842793,0.60587772,0.29808486,0.32822818,0.08243737]
m_qg = [0.50634945,0.24922782,0.40886978,0.50535604,0.34457819,0.35924829,0.10556061]
l_qg = [0.50454017,0.22878326,0.44996747,0.51033139,0.35635912,0.29376844,0.12998793]

In [220]:
l_pop = [l_pop]
l_qg = [l_qg]
l_pop_nor = preprocessing.normalize(l_pop, norm='l2')
l_qg_nor = preprocessing.normalize(l_qg, norm='l2')
print(l_pop_nor)
print(l_qg_nor)

[[0.44427431 0.06957249 0.53847435 0.68950068 0.15939147 0.08172322
  0.01816229]]


In [ ]:
m_pop = [m_pop]
m_qg = [m_qg]
m_pop_nor = preprocessing.normalize(m_pop, norm='l2')
m_qg_nor = preprocessing.normalize(m_qg, norm='l2')
print(m_pop_nor)
print(m_qg_nor)

In [ ]:
h_pop = [h_pop]
h_qg = [h_qg]
h_pop_nor = preprocessing.normalize(h_pop, norm='l2')
h_qg_nor = preprocessing.normalize(h_qg, norm='l2')
print(h_pop_nor)
print(h_qg_nor)

In [ ]:
import pandas as pd
import csv
hotel = []
n = 0
f = open(r'E:\selectedBrandAllInfo.csv',encoding='utf-8')
csv_read = csv.reader(f)
for line in csv_read:
    n += 1
    hotel.append(line)
f.close()
print(len(hotel))
print(n)

In [ ]:
df = pd.DataFrame(hotel, columns=['0', '1', 'hotelId','2','3','4','5','6','7','8','9','10','11','12','13','14','15','16','17','18','19']) 
df.head()

In [58]:
with open(r"E:\13万筛选三元组\修改\content_13w.txt", 'w',encoding='utf-8') as f:
    for i in content:
        f.write(str(i)+'\n')

In [52]:
high = []    
middle = []   
low = []

for i,y1 in enumerate(emotion_pairs):
    if(df.iloc[i]["hotelId"]==391657 or df.iloc[i]["hotelId"]==385119 or df.iloc[i]["hotelId"]==928851 or df.iloc[i]["hotelId"]==375079 or df.iloc[i]["hotelId"]==430265 or df.iloc[i]["hotelId"]==430270 or df.iloc[i]["hotelId"]==467314 or df.iloc[i]["hotelId"]==369688 or df.iloc[i]["hotelId"]==482983 or df.iloc[i]["hotelId"]==33587215 or df.iloc[i]["hotelId"]==427874 or df.iloc[i]["hotelId"]==53120643 or df.iloc[i]["hotelId"]==369708 or df.iloc[i]["hotelId"]==61840052 or df.iloc[i]["hotelId"]==347308 or df.iloc[i]["hotelId"]==347414):
        al[i].append(hotel[3])
        al[i].append(hotel[9])
        high.append(al[i])
    elif(df.iloc[i]["hotelId"]==430214 or df.iloc[i]["hotelId"]==427531 or df.iloc[i]["hotelId"]==430223 or df.iloc[i]["hotelId"]==747665 or df.iloc[i]["hotelId"]==430229 or df.iloc[i]["hotelId"]==980232 or df.iloc[i]["hotelId"]==46007521 or df.iloc[i]["hotelId"]==9619023 or df.iloc[i]["hotelId"]==63337143 or df.iloc[i]["hotelId"]==65394133 or df.iloc[i]["hotelId"]==9444856 or df.iloc[i]["hotelId"]==2272090 or df.iloc[i]["hotelId"]==48611068 or df.iloc[i]["hotelId"]==2231618 or df.iloc[i]["hotelId"]==763321 or df.iloc[i]["hotelId"]==11305146 or df.iloc[i]["hotelId"]==6589045 or df.iloc[i]["hotelId"]==65110141 or df.iloc[i]["hotelId"]==30006625 or df.iloc[i]["hotelId"]==12659788 or df.iloc[i]["hotelId"]==52159868 or df.iloc[i]["hotelId"]==32808664 or df.iloc[i]["hotelId"]==57019623 or df.iloc[i]["hotelId"]==60910621 or df.iloc[i]["hotelId"]==64226079 or df.iloc[i]["hotelId"]==1577568 or df.iloc[i]["hotelId"]==5754064 or df.iloc[i]["hotelId"]==5293044 or df.iloc[i]["hotelId"]==60976659 or df.iloc[i]["hotelId"]==37520955 or df.iloc[i]["hotelId"]==41648809):
        low.append(al[i])
    else:
        middle.append(al[i])

IndexError: single positional indexer is out-of-bounds

In [58]:
print(high[3])

[1, 1.5, 1, 1.0, 2, 3.0, 2, 2.5, 0, 0, 0, 0, 0, 0]


In [ ]:
import csv
data = []
f = open(r'D:\selectedBrandAllInfo.csv',encoding='utf-8')
csv_read = csv.reader(f)
index=0
for line in csv_read:
    if index != 0 :
        data.append(fx[index-1][::-1]+zx[index-1][::-1])                
        data[index-1].append(line[3])
        data[index-1].append(line[2])
        data[index-1].append(line[1])
        data[index-1].append(line[0])
    index+=1
f.close()

In [ ]:
rows=['hotelID','content','emotion_pairs','Factor1Count','Factor1Val','Factor2Count','Factor2Val','Factor3Count','Factor3Val','Factor4Count','Factor4Val','Factor5Count','Factor5Val']
f = open(r'E:\high_cot_val.csv','w',newline="")
writer = csv.writer(f)
writer.writerow(rows)
for l in high:
    writer.writerow(l)   
f.close()